# Preparation

In [4]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../../../../') # The simulator package and package needed for analysis 
                                  # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

In [ ]:
vehicle_rate = 45

## Simulation setup

### Simulation params setup

In [5]:
# Specify the labels of the road segments in each route
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [7]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

## For the connections of the topology, we can represent with concrete lines, but
# the vehicles will not actually go through these routes; 
# they are  for demonstration purposes
connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [8]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 0

# Utility interpolation factor between money and time
gamma_mean, gamma_var = 5, 0.5

# Utility exponent
eta_mean, eta_var = 0.5, 0.1

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 3min 31s, sys: 5.37 s, total: 3min 36s
Wall time: 3min 32s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]"
1,1,"[2, 1]",36.45,"[-999, -999]","[0, 0]","[0, 0]"
2,2,"[2, 3]",70.35,"[1, -999]","[0, 0]","[0, 0]"
3,3,"[0, 3]",106.05,"[0, 22]","[2.6999999999999984, 0]","[2.6999999999999984, 0]"
4,4,"[2, 3]",73.9,"[2, 2]","[0, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[2, 1]",190.5,"[72, 64]","[36.70000000000005, 0]","[0, 0]"
76,76,"[2, 1]",195.15,"[75, 75]","[41.49999999999978, 0]","[0, 0]"
77,77,"[0, 3]",263.2,"[74, 74]","[4.799999999999991, 0]","[0, 0]"
78,78,"[0, 1]",236.85,"[77, -999]","[5.249999999999989, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]",1
1,1,"[2, 1]",36.45,"[-999, -999]","[0, 0]","[0, 0]",1
2,2,"[2, 3]",70.35,"[1, -999]","[0, 0]","[0, 0]",1
3,3,"[0, 3]",106.05,"[0, 22]","[2.6999999999999984, 0]","[2.6999999999999984, 0]",1
4,4,"[2, 3]",73.9,"[2, 2]","[0, 0]","[0, 0]",1
5,5,"[0, 3]",108.3,"[3, 3]","[0, 0]","[0, 0]",1
6,6,"[2, 3]",76.65,"[4, 4]","[0, 0]","[0, 0]",1
7,7,"[0, 3]",115.4,"[5, 24]","[2.499999999999999, 0]","[2.499999999999999, 0]",1
8,8,"[0, 1]",87.2,"[7, -999]","[0, 0]","[0, 0]",1
9,9,"[0, 3]",122.55,"[8, 26]","[0, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.DataFrame(columns=['Vehicle_label', 'Utilities', 'Probabilities'])

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

### Sample values for the utility exponent $\eta$ and the utillity interpolation factor $\gamma$

In [14]:
etas = []
gammas = []

for i in range(len(utility_df)):
    eta = np.random.normal(eta_mean, eta_var, 1)[0]
    while eta < 0:
        eta = np.random.normal(eta_mean, eta_var, 1)[0]
    etas.append(eta)
    
for i in range(len(utility_df)):
    gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    while gamma < 0:
        gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    gammas.append(gamma)

utility_df['Gamma'] = gammas
utility_df['Eta'] = etas

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.310139,0.517878
1,1,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.825830,0.568200
2,2,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.590759,0.308298
3,3,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.157208,0.564622
4,4,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.026788,0.383511


## Summary of the 1st round

In [15]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,1,2.7,1,67.45
1,1,3,0.0,1,36.45
2,2,2,0.0,1,70.35
3,3,0,0.0,1,106.05
4,4,2,0.0,1,73.90
5,5,0,2.5,1,108.30
6,6,2,0.0,1,76.65
7,7,0,0.0,1,115.40
8,8,1,0.0,1,87.20
9,9,0,0.0,1,122.55


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[16.651092223153956, -0.2873080873937355, 16.6...","[0.33333332844113905, 1.4676582891824265e-08, ...",[1],5.310139,0.517878
1,1,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333332897275325, 0.33333332897275325, 0.3...",[3],4.825830,0.568200
2,2,"[16.651092223153956, 16.651092223153956, 0.245...","[0.3333333249944854, 0.3333333249944854, 2.501...",[2],4.590759,0.308298
3,3,"[-0.48414192465206596, 16.651092223153956, 16....","[1.205427523642938e-08, 0.33333332931524157, 0...",[0],5.157208,0.564622
4,4,"[16.651092223153956, 16.651092223153956, 0.085...","[0.33333332623250966, 0.33333332623250966, 2.1...",[2],5.026788,0.383511
5,5,"[-0.13288834984230324, 16.651092223153956, 16....","[1.712728760763978e-08, 0.33333332762423745, 0...",[0],4.988453,0.462755
6,6,"[16.651092223153956, 16.651092223153956, -0.25...","[0.3333333282614533, 0.3333333282614533, 1.521...",[2],4.878581,0.506162
7,7,"[-0.5129486351517298, 16.651092223153956, 16.6...","[1.1711985020849672e-08, 0.3333333294293383, 0...",[0],4.805346,0.571507
8,8,"[16.651092223153956, -0.3329282539891407, 16.6...","[0.3333333286593075, 1.4022077574522724e-08, 0...",[1],4.348500,0.530272
9,9,"[-0.011350678507525247, 16.651092223153956, 16...","[1.9340679566373167e-08, 0.33333332688644013, ...",[0],5.786703,0.416448


In [16]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [17]:
%%time

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)

CPU times: user 2h 39min 9s, sys: 4min 9s, total: 2h 43min 18s
Wall time: 2h 37min 38s
